In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
from random import randrange

from qiskit import IBMQ, BasicAer, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

from qiskit.visualization import plot_histogram

In [38]:
import csv

def oracul(inputText):    
    csv_path = "letters.csv"
    with open(csv_path, "r") as f_obj:
        reader = csv.reader(f_obj)
        letters = {}
        for row in reader:
            letters[row[1]] = row[0]
    #print(letters)

    csv_path = "pairLetters.csv"

    with open(csv_path, "r") as f_obj:
        reader = csv.reader(f_obj)
        pairLetters = {}
        for row in reader:
            i = 0
            for data in row: 
                if i % 2 == 0:
                    data1 = data
                else:
                    pairLetters[data1] = data
                i += 1
    #print(pairLetters)
    inputText = inputText.lower()

    #print(inputText)

    j = 0
    
    alphaCount = 0
    
    targetText = ""

    for symbol in inputText:
        if ((symbol >= "a") and (symbol <= "z")):
            alphaCount += 1
        
        if ((symbol < "a") or (symbol > "z")) and (symbol != " ") and (symbol != ",") and (symbol != ".") and (symbol != ":") and (symbol != ";") and (symbol != "!") and (symbol != "?") and (symbol != '"') and (symbol != "'") and (symbol != "№") and  not ((symbol >= '0') or (symbol <= '9') ):
            j += 1
        else:
            targetText += symbol
            
    leng = len(inputText)
    
    if  j / leng > 0.2:
        return 0
        
    
    targetText = targetText.upper()
    
    #print(targetText)
    
    
    res = {i : targetText.count(i)*100/alphaCount for i in letters}
    #print(res)
    
    pairCount = 0
    
    for i in pairLetters:
        pairCount += targetText.count(i)
    
    res1 = {i : targetText.count(i)*100/pairCount for i in pairLetters}
    
    #print(res1)
    i = 0
    for symbol in letters:
        i += abs(float(letters[symbol]) - float(res[symbol]))
    print("Absolute error(one letter): {:.2f}".format(i))
    if i > 30:
        print("Probability: 0")
        return 0
    
    i = 0
    for symbol in pairLetters:
        i += abs(float(pairLetters[symbol]) - float(res1[symbol]))
        
    print("Absolute error(bigrams):    {:.2f}".format(i))
    if i > 50:
        print("Probability: 0")
        return 0
    print("Probability: 1")
    return 1

In [2]:
import numpy as np
from Crypto.Cipher import DES
 
#key_target = np.random.randint(10000000, 99999999)
key_target = 12345678
print("Target key is", key_target)
key_target = str.encode(str(key_target))
 
def pad(text):
    while len(text) % 8 != 0:
        text += b' '
    return text
 
des_target = DES.new(key_target, DES.MODE_ECB)
text = open('Shakespeare William. The Sonnets.txt', 'rb').read().replace(b'\n', b' ').replace(b'\r', b' ')
padded_text = pad(text)
 
encrypted_text = des_target.encrypt(padded_text)
print("Text encrypted")

Target key is 12345678
Text encrypted


In [39]:
_ = decrypt_text(12345671)

Absolute error(one letter): 71.08
Probability: 0


In [40]:
_ = decrypt_text(12345678)

Absolute error(one letter): 13.55
Absolute error(bigrams):    37.02
Probability: 1


In [6]:
def decrypt_text(key):
#     key = 68694034     #from algorithm
    # print("Is key", key)
    key = str.encode(str(key))
    des = DES.new(key, DES.MODE_ECB)

    data = des.decrypt(encrypted_text) # to oracle
    data = data.decode('latin-1')
    
    return oracul(data)

def run_grovers_algorthm(encrypted_text):
    num_qbits = 8
    # number of oracles
    k = 1

    # Set n qubits and n bits
    qr = QuantumRegister(num_qbits)
    cr = ClassicalRegister(num_qbits)

    if num_qbits > 3:
        aux = QuantumRegister(num_qbits-3)
        groverCircuit = QuantumCircuit(qr,cr, aux)
    else:
        aux = None
        groverCircuit = QuantumCircuit(qr,cr)
        
    groverCircuit.h(qr)
    
    result = math.pi*math.sqrt((2**num_qbits)/k)/4 
iterations = round(result)
print("Number of inerations", iterations)

for i in range(iterations-1):
    
    
    groverCircuit.barrier(qr)
    
    groverCircuit.x(qr[1])
    groverCircuit.h(qr[num_qbits-1])
    groverCircuit.mct(qr[0:num_qbits-1], qr[num_qbits-1], aux)
    groverCircuit.h(qr[num_qbits-1])
    groverCircuit.x(qr[1])  

    groverCircuit.barrier(qr)
    groverCircuit.h(qr)
    groverCircuit.x(qr)
    groverCircuit.h(qr[num_qbits-1])
    groverCircuit.mct(qr[0:num_qbits-1], qr[num_qbits-1], aux)
    groverCircuit.h(qr[num_qbits-1])
    groverCircuit.x(qr)
    groverCircuit.h(qr)

groverCircuit.barrier(qr)

groverCircuit.draw(output="mpl")

b'12345678'
